In [1]:
import pandas
import numpy as np
import pylab as plt
import scipy
from scipy.optimize import curve_fit
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2021/bin/x86_64-linux'
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [2]:
plt.style.use('bmh')
plt.rc('text', usetex=True)

In [3]:
font = {'family':'serif','size':30, 'serif': ['computer modern roman']}
plt.rc('font',**font)

In [4]:
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
plt.rc('axes', grid=False, facecolor='white')

In [5]:
#Here, we are using a simple RIP model (S --> I --> P with rate constants ks and ki, respectively)
#S stands for substrate, I for intermediate, and P for product
#Test various ki and ks (with the constraint that ks > ki)
#Test also various tmin and tmax for finding different fitting windows
#Hopefelly, you will be convinced that as long as ks > ki, there will be a fitting window for fitting ki
#The larger is ks relative to ki, the better and more accurate is the fit, and the larger is the fitting window
#However, even when ki = 1 and ks = 1.001, the fitted ki output in the  window tmin = 3.5 and tmax = 10 
#is still decent
#The RIP model is not an exact match to our model, but it's a good approximation in the large [E]0 regime
#The fact that ks > ki holds (is not >>), our Eq S22 is valid

ki = 1
ks = 1.001

#Make a time array (sample array of 10 min, where time points were taken every 20 seconds)
def generate_time_array():
    return np.arange(0, 600 + 20, 20) / 60

t = generate_time_array()

#Equations from the RIP model
P = 1-((ki*np.exp(-ks*t)-ks*np.exp(-ki*t))/(ki-ks))

#We gave P_std some error; this can also be modulated (see how SD error on the fitted k_I changes)
P_std = np.full(len(P), 0.03) 

#Equations from the RIP model
S = np.exp(-ks*t) 
I = (ks/(ki-ks))*((np.exp(-ks*t)-np.exp(-ki*t)))
overlay_search = ki*I-ks*S

#Defining the windown of time in which kI will be fit
#To find a fitting window, choose tmin where the red and the green curve are close  
tmin = 3.5
tmax = 10

plt.axvline(x=tmin,linestyle='dashed',color='black',alpha=0.2)
plt.axvline(x=tmax,linestyle='dashed',color='black',alpha=0.2)

plt.plot(t,P,'k-',linewidth=2.5)
plt.plot(t,P+P_std,'k-',linewidth=1,alpha=0.3,label='STD')
plt.plot(t,P-P_std,'k-',linewidth=1,alpha=0.3)#,label='_nolegend_')
plt.plot(t,S,'b-',linewidth=2.5, label='S')
plt.plot(t,I,'g-',linewidth=2.5, label='I')
plt.plot(t,overlay_search,'r-',linewidth=2.5, label='kiI-ksS')
plt.xlabel('Time (s)')
plt.ylabel('[P]t (uM)')
plt.title('[P]t with STD error vs time')
plt.legend(fontsize='x-small')
plt.ylim(0, None)
x_ticks = np.arange(min(t), max(t) + 1, 1) 
plt.xticks(x_ticks)

# window of time that will be used for the fit
window = (t>tmin)*(t<tmax)

In [6]:
### Definining the formula [P]t = [P]inf - [I]tmin*e^(-ki*(t-tmin)) with three free fitting parameters
def func(t,Pinf,Itmin,kI): return Pinf - Itmin*np.exp(-kI*(t-tmin)) 

#Fitting the three independent parameters using curve_fit
#p0=[10,10,0.01] are the initial guesses as to what Pinf,Itmin, and kI are 10 uM, 10 uM, and 0.01 s-1, respectively
#The output is not too sensitive to initial guesses
parameters = curve_fit(func,t[window],P[window],p0=[0.9,0.3,0.5],sigma=P_std[window],absolute_sigma=True)
Pinf,Itmin,kI = parameters[0]

#parameters[1] contains the covariance matrix; square root of the diagonal values are standard deviations
Pinf_SD,Itmin_SD,kI_SD = np.sqrt(np.diag(parameters[1]))

print("original k_I we chose =", ki)
print("fitted k_I =", kI)
print("SD error on the fitted k_I =", kI_SD)

plt.show()

original k_I we chose = 1
fitted k_I = 0.8097169220802756
SD error on the fitted k_I = 0.3992360294734046
